# Ансамблевые методы: Random Forest, AdaBoost и Gradient Boosting

В этом notebook мы разберём три главных ансамблевых метода:
- Random Forest (параллельное обучение)
- AdaBoost (адаптивный бустинг)
- Gradient Boosting (градиентный бустинг)

Для каждого метода покажем:
- Как работает на синтетических и реальных данных
- Влияние гиперпараметров
- Важность признаков
- Сравнение скорости и точности

In [ ]:
# Импорты
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

# Ансамбли
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

# Визуализация
import time
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

np.random.seed(42)

---
## Часть 1: Random Forest

### Теория

Random Forest использует два механизма для снижения переобучения:
1. **Bagging:** каждое дерево обучается на bootstrap выборке
2. **Случайные признаки:** на каждом разбиении рассматривается только подмножество признаков

In [ ]:
# Создаём синтетические данные
X, y = make_classification(n_samples=1000, n_features=20, n_informative=10,
                          n_redundant=5, n_classes=2, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"Размер обучающей выборки: {X_train.shape}")
print(f"Размер тестовой выборки: {X_test.shape}")
print(f"Распределение классов: {np.bincount(y_train)}")

In [ ]:
# Влияние количества деревьев (n_estimators)
n_trees_range = np.array([1, 5, 10, 25, 50, 100, 200, 500])
train_scores = []
test_scores = []

for n_trees in n_trees_range:
    rf = RandomForestClassifier(n_estimators=n_trees, random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)
    
    train_score = rf.score(X_train, y_train)
    test_score = rf.score(X_test, y_test)
    
    train_scores.append(train_score)
    test_scores.append(test_score)
    
    print(f"n_trees={n_trees:3d}: train={train_score:.3f}, test={test_score:.3f}")

# Визуализация
plt.figure(figsize=(12, 6))
plt.plot(n_trees_range, train_scores, marker='o', label='Обучающая выборка')
plt.plot(n_trees_range, test_scores, marker='s', label='Тестовая выборка')
plt.xscale('log')
plt.xlabel('Количество деревьев')
plt.ylabel('Accuracy')
plt.title('Влияние количества деревьев на Random Forest')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Влияние max_depth на переобучение
depths = [1, 2, 3, 5, 10, 20, None]
train_scores_depth = []
test_scores_depth = []

for depth in depths:
    rf = RandomForestClassifier(n_estimators=100, max_depth=depth, 
                              random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)
    
    train_score = rf.score(X_train, y_train)
    test_score = rf.score(X_test, y_test)
    
    train_scores_depth.append(train_score)
    test_scores_depth.append(test_score)
    
    depth_label = 'None' if depth is None else str(depth)
    print(f"max_depth={depth_label:4s}: train={train_score:.3f}, test={test_score:.3f}")

# Визуализация
depths_str = [str(d) if d is not None else 'None' for d in depths]
x_pos = np.arange(len(depths_str))

plt.figure(figsize=(12, 6))
plt.plot(x_pos, train_scores_depth, marker='o', label='Обучающая выборка')
plt.plot(x_pos, test_scores_depth, marker='s', label='Тестовая выборка')
plt.xticks(x_pos, depths_str)
plt.xlabel('max_depth')
plt.ylabel('Accuracy')
plt.title('Влияние max_depth на Random Forest (переобучение)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Out-of-Bag (OOB) ошибка
rf_oob = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=42)
rf_oob.fit(X_train, y_train)

oob_score = rf_oob.oob_score_
test_score = rf_oob.score(X_test, y_test)

print(f"OOB Score (на обучающей выборке): {oob_score:.3f}")
print(f"Test Score (на отдельной тестовой): {test_score:.3f}")
print(f"Разница: {abs(oob_score - test_score):.3f}")

print("\nOOB даёт хорошую оценку обобщающей способности без отдельной тестовой выборки!")

In [ ]:
# Feature Importance
rf_best = RandomForestClassifier(n_estimators=100, random_state=42)
rf_best.fit(X_train, y_train)

importances = rf_best.feature_importances_
indices = np.argsort(importances)[-10:]  # Топ-10 признаков

plt.figure(figsize=(12, 6))
plt.barh(range(len(indices)), importances[indices])
plt.yticks(range(len(indices)), [f'Feature {i}' for i in indices])
plt.xlabel('Важность признака')
plt.title('Топ-10 важных признаков в Random Forest')
plt.show()

print(f"Сумма важностей: {importances.sum():.3f}")
print(f"Топ-3 признака: {indices[-3:]} с важностями {importances[indices[-3:]]}")

---
## Часть 2: AdaBoost

### Теория

AdaBoost использует адаптивные веса: примеры, которые сложно классифицировать, получают больший вес на следующей итерации.

In [ ]:
# Сравнение AdaBoost с базовой моделью
base_clf = DecisionTreeClassifier(max_depth=1)  # "пень" (weak learner)

adaboost = AdaBoostClassifier(estimator=base_clf, n_estimators=100, 
                             random_state=42)
adaboost.fit(X_train, y_train)

base_score = base_clf.fit(X_train, y_train).score(X_test, y_test)
ada_score = adaboost.score(X_test, y_test)

print(f"Базовая модель (пень): {base_score:.3f}")
print(f"AdaBoost (100 пней): {ada_score:.3f}")
print(f"Улучшение: {(ada_score - base_score):.3f}")

In [ ]:
# Влияние количества итераций на AdaBoost
n_estimators_range = np.array([1, 5, 10, 20, 50, 100, 200])
ada_train_scores = []
ada_test_scores = []

for n_est in n_estimators_range:
    base_clf = DecisionTreeClassifier(max_depth=1)
    ada = AdaBoostClassifier(estimator=base_clf, n_estimators=n_est, 
                           random_state=42)
    ada.fit(X_train, y_train)
    
    train_score = ada.score(X_train, y_train)
    test_score = ada.score(X_test, y_test)
    
    ada_train_scores.append(train_score)
    ada_test_scores.append(test_score)
    
    print(f"n_est={n_est:3d}: train={train_score:.3f}, test={test_score:.3f}")

# Визуализация
plt.figure(figsize=(12, 6))
plt.plot(n_estimators_range, ada_train_scores, marker='o', label='Обучающая выборка')
plt.plot(n_estimators_range, ada_test_scores, marker='s', label='Тестовая выборка')
plt.xscale('log')
plt.xlabel('Количество итераций')
plt.ylabel('Accuracy')
plt.title('Влияние n_estimators на AdaBoost')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Feature Importance в AdaBoost
base_clf = DecisionTreeClassifier(max_depth=1)
ada_fi = AdaBoostClassifier(estimator=base_clf, n_estimators=100, random_state=42)
ada_fi.fit(X_train, y_train)

importances_ada = ada_fi.feature_importances_
indices_ada = np.argsort(importances_ada)[-10:]

plt.figure(figsize=(12, 6))
plt.barh(range(len(indices_ada)), importances_ada[indices_ada])
plt.yticks(range(len(indices_ada)), [f'Feature {i}' for i in indices_ada])
plt.xlabel('Важность признака')
plt.title('Топ-10 важных признаков в AdaBoost')
plt.show()

---
## Часть 3: Gradient Boosting

### Теория

Gradient Boosting каждое новое дерево учится предсказывать остатки (residuals) предыдущей модели.

In [ ]:
# Базовое Gradient Boosting
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, 
                              max_depth=3, random_state=42)
gb.fit(X_train, y_train)

gb_train_score = gb.score(X_train, y_train)
gb_test_score = gb.score(X_test, y_test)

print("Gradient Boosting:")
print(f"Train accuracy: {gb_train_score:.3f}")
print(f"Test accuracy: {gb_test_score:.3f}")

In [ ]:
# Влияние learning_rate
learning_rates = [0.001, 0.01, 0.05, 0.1, 0.2, 0.5]
gb_lr_train = []
gb_lr_test = []

for lr in learning_rates:
    gb = GradientBoostingClassifier(n_estimators=100, learning_rate=lr, 
                                  max_depth=3, random_state=42)
    gb.fit(X_train, y_train)
    
    train_score = gb.score(X_train, y_train)
    test_score = gb.score(X_test, y_test)
    
    gb_lr_train.append(train_score)
    gb_lr_test.append(test_score)
    
    print(f"lr={lr:.3f}: train={train_score:.3f}, test={test_score:.3f}")

# Визуализация
plt.figure(figsize=(12, 6))
plt.plot(learning_rates, gb_lr_train, marker='o', label='Обучающая выборка')
plt.plot(learning_rates, gb_lr_test, marker='s', label='Тестовая выборка')
plt.xscale('log')
plt.xlabel('Learning Rate')
plt.ylabel('Accuracy')
plt.title('Влияние learning_rate на Gradient Boosting')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Влияние max_depth (глубина деревьев)
depths_gb = [1, 2, 3, 4, 5, 8, 10]
gb_depth_train = []
gb_depth_test = []

for depth in depths_gb:
    gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, 
                                  max_depth=depth, random_state=42)
    gb.fit(X_train, y_train)
    
    train_score = gb.score(X_train, y_train)
    test_score = gb.score(X_test, y_test)
    
    gb_depth_train.append(train_score)
    gb_depth_test.append(test_score)
    
    print(f"depth={depth:2d}: train={train_score:.3f}, test={test_score:.3f}")

# Визуализация
plt.figure(figsize=(12, 6))
plt.plot(depths_gb, gb_depth_train, marker='o', label='Обучающая выборка')
plt.plot(depths_gb, gb_depth_test, marker='s', label='Тестовая выборка')
plt.xlabel('max_depth')
plt.ylabel('Accuracy')
plt.title('Влияние max_depth на Gradient Boosting')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Кривая обучения: как ошибка меняется с каждой итерацией
gb_learn = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, 
                                     max_depth=3, random_state=42)
gb_learn.fit(X_train, y_train)

train_errors = []
test_errors = []

for i, y_pred in enumerate(gb_learn.staged_predict(X_train)):
    train_error = 1 - accuracy_score(y_train, y_pred)
    train_errors.append(train_error)

for i, y_pred in enumerate(gb_learn.staged_predict(X_test)):
    test_error = 1 - accuracy_score(y_test, y_pred)
    test_errors.append(test_error)

plt.figure(figsize=(12, 6))
plt.plot(train_errors, label='Ошибка на обучении', alpha=0.7)
plt.plot(test_errors, label='Ошибка на тесте', alpha=0.7)
plt.xlabel('Номер итерации (дерева)')
plt.ylabel('Ошибка (1 - Accuracy)')
plt.title('Кривая обучения Gradient Boosting')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---
## Часть 4: Сравнение всех трёх методов

In [ ]:
# Обучение всех моделей
models = {
    'Single Tree': DecisionTreeClassifier(max_depth=5, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'AdaBoost': AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                                 n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,
                                                   max_depth=3, random_state=42)
}

results = []

for name, model in models.items():
    # Время обучения
    start = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start
    
    # Время предсказания
    start = time.time()
    y_pred = model.predict(X_test)
    pred_time = time.time() - start
    
    # Метрики
    train_acc = model.score(X_train, y_train)
    test_acc = model.score(X_test, y_test)
    
    results.append({
        'Model': name,
        'Train Acc': train_acc,
        'Test Acc': test_acc,
        'Overfit': train_acc - test_acc,
        'Train Time (s)': train_time,
        'Pred Time (ms)': pred_time * 1000
    })
    
    print(f"\n{name}:")
    print(f"  Train accuracy: {train_acc:.3f}")
    print(f"  Test accuracy: {test_acc:.3f}")
    print(f"  Overfit gap: {train_acc - test_acc:.3f}")
    print(f"  Train time: {train_time:.3f}s")
    print(f"  Pred time: {pred_time*1000:.2f}ms")

results_df = pd.DataFrame(results)

In [ ]:
# Визуализация сравнения
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Accuracy
x = np.arange(len(results_df))
width = 0.35
axes[0, 0].bar(x - width/2, results_df['Train Acc'], width, label='Train', alpha=0.8)
axes[0, 0].bar(x + width/2, results_df['Test Acc'], width, label='Test', alpha=0.8)
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].set_title('Точность моделей')
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(results_df['Model'], rotation=45)
axes[0, 0].legend()
axes[0, 0].grid(axis='y', alpha=0.3)

# Overfit gap
axes[0, 1].bar(results_df['Model'], results_df['Overfit'], color='salmon')
axes[0, 1].set_ylabel('Train Acc - Test Acc')
axes[0, 1].set_title('Переобучение (зазор между train/test)')
axes[0, 1].set_xticklabels(results_df['Model'], rotation=45)
axes[0, 1].grid(axis='y', alpha=0.3)

# Время обучения
axes[1, 0].bar(results_df['Model'], results_df['Train Time (s)'], color='skyblue')
axes[1, 0].set_ylabel('Время (сек)')
axes[1, 0].set_title('Время обучения')
axes[1, 0].set_xticklabels(results_df['Model'], rotation=45)
axes[1, 0].grid(axis='y', alpha=0.3)

# Время предсказания
axes[1, 1].bar(results_df['Model'], results_df['Pred Time (ms)'], color='lightgreen')
axes[1, 1].set_ylabel('Время (мс)')
axes[1, 1].set_title('Время предсказания')
axes[1, 1].set_xticklabels(results_df['Model'], rotation=45)
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\nСводная таблица:")
print(results_df.to_string(index=False))

---
## Часть 5: Grid Search для оптимизации гиперпараметров

In [ ]:
# Grid Search для Gradient Boosting
param_grid_gb = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [2, 3, 4],
    'subsample': [0.7, 0.8, 1.0]
}

print("Начинаем Grid Search для Gradient Boosting...")
print(f"Всего комбинаций: {np.prod([len(v) for v in param_grid_gb.values()])}")

grid_search = GridSearchCV(GradientBoostingClassifier(random_state=42),
                         param_grid_gb, cv=5, scoring='accuracy', n_jobs=-1)

start = time.time()
grid_search.fit(X_train, y_train)
search_time = time.time() - start

print(f"\nЛучшие параметры: {grid_search.best_params_}")
print(f"Лучшая CV точность: {grid_search.best_score_:.3f}")

best_gb = grid_search.best_estimator_
test_score = best_gb.score(X_test, y_test)
print(f"Тестовая точность: {test_score:.3f}")
print(f"Время Grid Search: {search_time:.2f}s")

In [ ]:
# Анализ результатов Grid Search
results_cv = pd.DataFrame(grid_search.cv_results_)
results_cv['rank_test_score'] = results_cv['rank_test_score']

# Топ-5 комбинаций
top5 = results_cv.nsmallest(5, 'rank_test_score')[['param_n_estimators', 'param_learning_rate', 
                                                   'param_max_depth', 'param_subsample', 
                                                   'mean_test_score']]
print("Топ-5 комбинаций гиперпараметров:")
print(top5.to_string(index=False))

---
## Итоговые выводы

### Когда использовать каждый метод:

**Random Forest:**
- ✅ Быстро обучается
- ✅ Хорошие результаты без подбора
- ✅ Легко интерпретировать (Feature Importance)
- ❌ Может быть медленнее на очень больших данных

**AdaBoost:**
- ✅ Исторически важна (основа современного бустинга)
- ✅ Хороша на чистых данных
- ✅ Часто работает из коробки
- ❌ Чувствительна к выбросам
- ❌ Последовательная (не параллелизируется)

**Gradient Boosting:**
- ✅ Обычно лучшая точность
- ✅ Early stopping предотвращает переобучение
- ✅ Гибкая (любая дифференцируемая функция потерь)
- ❌ Требует подбора гиперпараметров
- ❌ Медленнее обучается

### Практический совет:

1. Начните с Random Forest (быстро, хорошо)
2. Если нужна лучше точность → Gradient Boosting
3. Используйте Grid Search для оптимизации
4. Проверяйте на кросс-валидации, не переобучайтесь!